In [217]:
import pandas as pd 
import numpy as np 
import tabula

In [2]:
pdf_file = 'Cebu.Survey.Report[final].pdf'
df1 = tabula.read_pdf(pdf_file, pages = "21")
df2 = tabula.read_pdf(pdf_file, pages = "22")
df3 = tabula.read_pdf(pdf_file, pages = "23")

Got stderr: May 01, 2021 7:22:02 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
May 01, 2021 7:22:02 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
May 01, 2021 7:22:02 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
May 01, 2021 7:22:02 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>

Got stderr: May 01, 2021 7:22:05 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>

Got stderr: May 01, 2021 7:22:09 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>



In [97]:
def identify_null_pct(srs):
    num_na = srs.isna().sum()
    return num_na/srs.shape[0] < 0.5


In [214]:
def preprocess(df):
    #identify object columns
    df = df[2].copy()
    obj_cols = df.columns[df.dtypes == 'object']
    # identify the column that contains '/H/' this contains the names of the households
    h_col = obj_cols[df.loc[:, obj_cols].apply(lambda x: x.str.contains('/H').any())]
    # only consider columns that are > 10% non-null
    non_nan_cols = df.columns[df.apply(identify_null_pct)]
    fnl = df.loc[:, non_nan_cols]
    # remove last two columns (kg per day and g per day)
    fnl = fnl.iloc[:,:-2]
    return fnl


    

In [215]:
preprocess(df3)

,Unnamed: 1,Unnamed: 4,Unnamed: 6,Unnamed: 8,Unnamed: 10,Unnamed: 12,Unnamed: 14,Unnamed: 16
0,amputhaw,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,/H/1,5,6.5,1.2,2.00,0.0,0.00,9.7
2,/H/2,5,3.0,2.0,2.50,0.0,0.00,8.0
3,/H/3,2,6.0,1.7,1.25,0.0,0.00,9.0
4,/H/4,5,2.0,NaN,0.00,0.0,0.00,2.0
5,/H/5,4,6.2,2.0,2.50,0.0,0.00,11.2
6,/H/6,2,2.0,2.0,2.50,0.0,0.00,7.0
7,/H/7,6,6.0,2.0,2.60,0.0,0.00,11.0
8,/H/8,3,2.7,2.0,2.50,2.0,0.00,9.7
9,/H/9,9,0.0,NaN,0.00,0.0,3.00,3.0


We then process each dataframe one by one since there are some nuances in the data structure. For df1, we just replace the columns with the real column names.

In [209]:
df_proc_dict = {i : preprocess(df).reset_index(drop = True) for i, df in enumerate([df1, df2, df3])}
# specify the column names and data types
cols = {'hh_id': "str", 
        'hh_members': "int32", 
        'organic': "float64",
        'paper': "float64",
        'plastic': "float64", 
        'metal': "float64",
        'others': "float64", 
        'total': "float64"}
# for df1 and df3 rename columns
df_proc_dict[0].columns = list(cols.keys())

df_proc_dict[2].columns = list(cols.keys())
df_proc_dict[0] = df_proc_dict[0].astype(dtype = cols)
df_proc_dict[2] = df_proc_dict[2].astype(dtype = cols)
# and remove whitespace for hh_id column in df1
df_proc_dict[0]['hh_id'] = df_proc_dict[0]['hh_id'].str.replace(' ', '').str.strip()
# add H/ in df2 and df3
df_proc_dict[2]['hh_id'] = ['H' + i for i in df_proc_dict[2]['hh_id'].str.strip()]


For df2, the first row seems to be real data. We extract this and put this at the top of the dataframe.

In [210]:
# for df2, add the column name as the first row.
tmp = df_proc_dict[1].copy()
tmp.head()

,/H/12,5,2,1.,1.25,0,2.75,7.5
0,/H/13,6.0,1.9,0.0,2.00,0.0,0.75,5.1
1,/H/14,2.0,2.3,0.1,1.15,0.0,0.00,3.6
2,/H/15,1.0,1.7,0.6,0.65,0.2,0.75,4.0
3,/H/16,7.0,7.7,0.2,0.80,0.0,0.80,9.6
4,/H/17,4.0,1.2,0.8,0.50,0.0,0.85,3.4


In [211]:
row1 = list(tmp.columns)
# _dict = {col: dt for col, dt in zip(cols, row1)}
tmp.columns = cols
tmp.loc[-1] = row1
tmp.index = tmp.index + 1
tmp = tmp.sort_index()
df_proc_dict[1] = tmp.astype(cols)
# add H in hh_id
df_proc_dict[1]['hh_id'] = ['H' + i for i in df_proc_dict[1]['hh_id']]
tmp.head()

,hh_id,hh_members,organic,paper,plastic,metal,others,total
0,/H/12,5,2,1.,1.25,0,2.75,7.5
1,/H/13,6.0,1.9,0.0,2.0,0.0,0.75,5.1
2,/H/14,2.0,2.3,0.1,1.15,0.0,0.0,3.6
3,/H/15,1.0,1.7,0.6,0.65,0.2,0.75,4.0
4,/H/16,7.0,7.7,0.2,0.8,0.0,0.8,9.6


In [212]:
pd.concat([v for _, v in df_proc_dict.items()])

,hh_id,hh_members,organic,paper,plastic,metal,others,total
0,H/H/1,7,5.2,2.2,2.25,0.0,0.00,9.7
1,H/H/2,4,6.5,2.0,2.25,0.0,5.00,16.2
2,H/H/10,6,6.0,1.2,2.00,0.0,3.00,12.2
3,H/H/11,8,5.5,3.1,2.00,0.0,0.00,10.6
4,H/H/12,9,5.0,2.0,2.50,0.0,0.00,1.0
...,...,...,...,...,...,...,...,...
26,H/H/14,2,0.5,0.7,0.50,0.0,2.50,4.2
27,H/H/15,3,1.0,1.0,0.50,0.7,1.75,5.3
28,H/H/16,5,1.5,0.1,1.30,0.0,1.25,4.2
29,H/H/17,4,0.7,0.0,0.15,0.1,0.80,2.3


It seems that the above method misses out alot of things, I just opted now to copy paste in a txt file the data, and do the processing from the txt file

In [219]:
pd.read_csv("cebu_hh_survey_2011.txt", delim_whitespace=True)

,barangay,hh_id,hh_size,organic,paper,plastic,metal,others,total,av_kg_per_day,av_g_per_day
0,Capitol_Site,H/H/1,7,5.25,2.25,2.25,0.00,0.00,9.75,1.39,"1,390"
1,Capitol_Site,H/H/2,4,6.50,2.50,2.25,0.00,5.00,16.25,2.32,"2,320"
2,Capitol_Site,H/H/3,6,6.50,2.00,2.00,0.00,7.00,17.50,2.50,"2,500"
3,Capitol_Site,H/H/4,3,8.25,2.00,2.25,0.00,0.00,12.50,1.79,"1,790"
4,Capitol_Site,H/H/5,9,9.00,2.00,2.00,0.00,0.00,13.00,1.86,"1,860"
...,...,...,...,...,...,...,...,...,...,...,...
113,Quiot,H/H/15,3,1.00,1.30,0.50,0.75,1.75,5.30,0.76,760
114,Quiot,H/H/16,5,1.50,0.15,1.30,0.00,1.25,4.20,0.60,600
115,Quiot,H/H/17,4,0.75,0.50,0.15,0.15,0.80,2.35,0.34,340
116,Quiot,H/H/18,6,3.30,0.60,1.30,0.75,3.00,8.95,1.28,"1,280"
